지니 Top200 차트를 selenium과 beautifulsoup을 활용해서 크롤링해보기

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
url= 'https://www.genie.co.kr/chart/top200'
headers={"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
res=requests.get(url, headers=headers)
soup=BeautifulSoup(res.text,'html.parser')

In [7]:
soup.select_one('#inc_date')

<span id="inc_date">2013.03.06</span>

In [8]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [9]:
driver=webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [38]:
date=soup.select_one('#inc_date')

In [40]:
time=soup.select_one('#inc_time')
time

<span id="inc_time">12:00</span>

In [41]:
driver.close()

In [42]:
trs=soup.select('tr.list')

In [44]:
tr=trs[0]
tr

<tr class="list" songid="104534602">
<td class="check"><input class="select-check" title="To. X" type="checkbox"/></td>
<td class="number">1
                                        
                                    <span class="rank">
<span class="rank"><span class="rank-none"><span class="hide">유지</span></span></span>
</span>
</td>
<td><a class="cover" href="#" onclick="fnViewAlbumLayer('84539814');return false;"><span class="mask"></span><img alt="To. X - The 5th Mini Album" onerror="this.src='//image.genie.co.kr/imageg/web/common/blank_68.gif';" src="//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/539/814/84539814_1701067108915_1_140x140.JPG/dims/resize/Q_80,0"/></a></td>
<td class="link"><a class="btn-basic btn-info" href="#" onclick="fnViewSongInfo('104534602');return false;">곡 제목 정보 페이지</a></td>
<td class="info">
<a class="title ellipsis" href="#" onclick="fnPlaySong('104534602','1');return false;" title="재생">
                                        
                                

In [47]:
rank=tr.select_one('.number').get_text().split('\n')[0].strip()
rank

'1'

In [48]:
title=tr.select_one('.title.ellipsis').get_text().strip()
title

'To. X'

In [32]:
artist=tr.select_one('.artist.ellipsis').get_text().strip()
artist

'태연 (TAEYEON)'

In [49]:
album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'To. X - The 5th Mini Album'

In [50]:
data=[]
for tr in trs:
    rank=tr.select_one('.number').get_text().split('\n')[0].strip()
    title=tr.select_one('.title.ellipsis').get_text().strip()
    artist=tr.select_one('.artist.ellipsis').get_text().strip()
    album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
    data.append({'rank':rank, 'title':title, 'artist':artist,'album':album})
data #첫페이지(50위까지)    

[{'rank': '1',
  'title': 'To. X',
  'artist': '태연 (TAEYEON)',
  'album': 'To. X - The 5th Mini Album'},
 {'rank': '2',
  'title': 'Drama',
  'artist': 'aespa',
  'album': 'Drama - The 4th Mini Album'},
 {'rank': '3',
  'title': 'Perfect Night',
  'artist': 'LE SSERAFIM (르세라핌)',
  'album': 'Perfect Night'},
 {'rank': '4', 'title': '헤어지자 말해요', 'artist': '박재정', 'album': '1집 Alone'},
 {'rank': '5', 'title': 'Baddie', 'artist': 'IVE (아이브)', 'album': "I'VE MINE"},
 {'rank': '6',
  'title': 'Love Lee',
  'artist': 'AKMU (악뮤)',
  'album': 'Love Lee'},
 {'rank': '7', 'title': '후라이의 꿈', 'artist': 'AKMU (악뮤)', 'album': 'Love Lee'},
 {'rank': '8', 'title': 'I AM', 'artist': 'IVE (아이브)', 'album': "I've IVE"},
 {'rank': '9',
  'title': 'You & Me',
  'artist': '제니 (JENNIE)',
  'album': 'JENNIE Special Single ［You & Me］'},
 {'rank': '10',
  'title': '사막에서 꽃을 피우듯',
  'artist': '우디 (Woody)',
  'album': '사막에서 꽃을 피우듯'},
 {'rank': '11',
  'title': 'ETA',
  'artist': 'NewJeans',
  'album': "NewJeans 2nd EP

In [51]:
len(data)

50

In [52]:
#나머지 페이지
from tqdm import tqdm
data=[]
for page in tqdm(range(1,5)):
    url= f'https://www.genie.co.kr/chart/top200'
    res=requests.get(url, headers=headers)
    soup=BeautifulSoup(res.text,'html.parser')
    trs=soup.select('tr.list')
    for tr in trs:
        rank=tr.select_one('.number').get_text().split('\n')[0].strip()
        title=tr.select_one('.title.ellipsis').get_text().strip()
        artist=tr.select_one('.artist.ellipsis').get_text().strip()
        album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
        data.append({'rank':rank, 'title':title, 'artist':artist,'album':album})

100%|██████████| 4/4 [00:01<00:00,  3.54it/s]


In [53]:
df=pd.DataFrame(data)
df.head()

,rank,title,artist,album
0,1,To. X,태연 (TAEYEON),To. X - The 5th Mini Album
1,2,Drama,aespa,Drama - The 4th Mini Album
2,3,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night
3,4,Love Lee,AKMU (악뮤),Love Lee
4,5,헤어지자 말해요,박재정,1집 Alone
